In [50]:
import numpy as np
from scipy.special import logsumexp
from numpy.linalg import matrix_power
from scipy.integrate import quad

# PS0 Empirical IO: Numerical Stuff

## Part 0: Logit Inclusive Value

### 1 - Convexity

The logit inclusive value is the log of the denominator of a logit probability of choice. It is as follows:

$$
IV = \log \sum_{i=0}^N \exp(x_i)
$$

Suppose $x_0 = 0$. Then, $IV(x_1, ... , x_n) = \log \sum_{i=1}^N (1+\exp(x_i))$. We want to show that this function is convex everywhere. The second derivative of this function is:
$$
\frac{\partial^2}{\partial x_i^2} \log\left(N + \sum_{j=1}^N \exp(x_j)\right) = \exp(2x_i) \left( \frac{S - 1}{S^2} \right)
 \\
S = N + \sum_{j=1}^N \exp(x_j).
$$

Which is indeed positive. Therefore, this function is convex.

### 2- Large $x_i$ problem

For large values of $x_i$, we cannot manage to find the exponential.

To resolve this issue, we now that, given $m = \max_i x_i$:

$$
IV = \log \sum_{i=0}^N \exp(x_i) = \log \left(\exp(m)\sum_{i=0}^N \exp(x_i - m)\right)
$$
Therefore, we have:

$$
IV = m + \log \sum_{i=0}^N \exp(x_i - m)
$$

In [7]:
def solve_IV(x):
    m = np.max(x)
    IV = m + np.log(np.sum(np.exp(x - m)))
    return IV

In [22]:
x = np.array([300, 400, 500, 750, 799, 800])
IV_stable = solve_IV(x)
print("Using the above method, IV is:", IV_stable)
IV_unstable = np.log(np.sum(np.exp(x)))
print("Using the naive method, IV is:", IV_unstable)

Using the above method, IV is: 800.3132616875182
Using the naive method, IV is: inf


C:\Users\moham\AppData\Local\Temp\ipykernel_24228\1108397953.py:4: RuntimeWarning: overflow encountered in exp
  IV_unstable = np.log(np.sum(np.exp(x)))


### 3. logsumexp function in python

In [24]:
IV_stable_python = logsumexp(x)
print("Using scipy's logsumexp, IV is:", IV_stable_python)

Using scipy's logsumexp, IV is: 800.3132616875182


This function in python uses the same method as explained above.

## Part 1: Markov Chains

Here, the goal is to write a function to solve for the ergodic distribution of a markov process, given its transition matrix. 

To do this properly, we can use the idea of eigenvalues and eigen vectors. We know that, the ergodic distribution is the fixed point of $\pi P$. Therefore:

$$
\pi = \pi P \rightarrow (P' - I)\pi' = 0
$$

So, the ergodic distribution is the normalized eigenvector (should be added to one) corresponding to eigenvalue of 1 for matrix $P'$.

In [43]:
import numpy as np

def ergodic_distribution(P):
    # Step 1: Compute the eigenvalues and eigenvectors of the transpose of P
    eigenvalues, eigenvectors = np.linalg.eig(P.T)
    
    # Step 2: Find the eigenvector corresponding to the eigenvalue 1
    # We assume there is exactly one eigenvalue that is 1 for a valid transition matrix
    eigenvector = eigenvectors[:, np.isclose(eigenvalues, 1)]
    
    # Step 3: Normalize the eigenvector
    steady_state = eigenvector[:, 0]  # Extract the eigenvector (as it's returned in a 2D array)
    steady_state = steady_state / np.sum(steady_state)
    
    # Ensure the result is real (numerical computations may result in a complex vector with very small imaginary parts)
    steady_state = steady_state.real
    
    return steady_state

In [44]:
P = np.array([[0.2, 0.4, 0.4],
              [0.1, 0.3, 0.6],
              [0.5, 0.1, 0.4]])
steady_state = ergodic_distribution(P)
print("The steady state distribution is:", steady_state.round(3))

The steady state distribution is: [0.31  0.241 0.448]


Now, notice that:
$$
\pi_t = \pi_0 P^t
$$
So, to find the ergodic distribution, we can set $\pi_0 = [1, 0, ..., 0]$ and find $\pi_t$ for large values of $t$.

In [49]:
pi_0 = np.array([1, 0, 0])
ergodic = pi_0 @ matrix_power(P, 100)
print("The ergodic distribution is:", ergodic.round(3))

The ergodic distribution is: [0.31  0.241 0.448]


One can see that these are the same results. When reached to the ergodic distribution, the distribution of transitioning from each alternative is the same and equal to the ergodic distribution.

## Part 2: Numerical Integration

Here, the goal is to solve for the logit choice probability by numerical integration. 

$$
p(X, \theta)=\int_{-\infty}^{\infty} \frac{\exp \left(\beta_i X\right)}{1+\exp \left(\beta_i X\right)} f\left(\beta_i \mid \theta\right) \partial \beta_i \,\, ,\,\, f\sim N(0.5,2) \,\, ,\,\, X = 0.5
$$

Using the quad function in python, we have:

In [74]:
X = 0.5
mean = 0.5
sigma = np.sqrt(2)
integral = quad(lambda beta: np.exp(beta*X)/(1 + np.exp(beta*X))
                * 1/(np.sqrt(2*np.pi) * sigma) * 
                np.exp(-((beta - mean)/sigma)**2/2), -100, 100, epsabs=1e-14)[0]
print("The integral is:", integral)

The integral is: 0.5559391628434652
